In [ ]:
import sys
sys.path.append("D:\\WETrak")

import os
import configparser
import numpy as np
import torch
from torch.utils.data import DataLoader

import import_ipynb
import branch_adapter.regression.regression_model_utils as utils
from branch_adapter.regression.regression_loss import mse_loss
from branch_adapter.regression.regression_network import MLP_regression
from branch_adapter.regression.regression_data import regression_data

In [ ]:
def train_epoch(model,train_loader,epoch,optimizer,lr,finger_type):
    losses,errors = utils.AverageMeter(),utils.AverageMeter()    
    
    model.train()
    
    for i,(emg,gt_regression,bin_lower_limit,bin_upper_limit) in enumerate(train_loader):
        emg = utils.data_gpu(emg, device)
        gt = gt_regression * (bin_upper_limit-bin_lower_limit)+bin_lower_limit 
        gt = utils.data_gpu(gt, device)
#         print("emg.shape",emg.shape)
#         print("gt.shape",gt.shape)
        out = model(emg) 
        weight = (bin_upper_limit-bin_lower_limit).to(device)
        bias = bin_lower_limit.to(device)
        out = out * weight + bias
#         print("out.shape",out.shape)
        
        loss = mse_loss(out,gt)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        error = utils.mean_absolute_error(gt, out)
        errors.update(error.item())
        losses.update(loss.item())

In [ ]:
def main(user_name,classify_num,config):
    data_path = config['main']['data_path']
    output_path = config['main']['output_path']
    
    EPOCHS = int(config['Regression']['epochs'])
    BATCH_SIZE = int(config['Regression']['batch_size'])
    lr = float(config['Regression']['learning_rate'])
    
    finger_type_list = ['Index','Middle','Ring']
    # -------------------------------------Save Dir initialization----------------------------------------------------- #
    runs_dir = os.path.join(output_path, user_name,"regression",str(classify_num))
    save_dir = os.path.join(runs_dir,'checkpoints')
    for t in (runs_dir,save_dir):
        if not(os.path.isdir(t)): os.makedirs(t)
    print("runs_dir",os.path.abspath(runs_dir))

    # -------------------------------------Gpu Device && Tensorboard--------------------------------------------------- #
    global device
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    # -------------------------------------Model and Optim Initialize-------------------------------------------------- #
    model = MLP_regression(output_size = 2*4*len(finger_type_list)).cuda()
    optimizer = torch.optim.Adam(model.parameters(),lr = lr, weight_decay=0.0005)

    # -------------------------------------Data Initialize--------------------------------------------------- #
    train_set = regression_data(data_path, user_name, "train", finger_type_list,classify_num)
    train_loader = DataLoader(train_set,batch_size= BATCH_SIZE,shuffle = True)

    # -------------------------------------Training--------------------------------------------------- #
    for epoch in range(EPOCHS): 
        train_epoch(model,train_loader,epoch,optimizer,lr,finger_type_list)

        if epoch%2 == 0:
            # save newest model:
            utils.save_checkpoint({'epoch': epoch, 'model_state_dict':model.state_dict(),
                'optimizer_state_dict':optimizer.state_dict(),'error':best_avg_error},save_dir,flag="newest")

In [ ]:
user_name = "user1"
classify_num = 15

# Read configuration
config = configparser.ConfigParser()
config.read('D:\\WETrak\\config.ini')

main(user_name,classify_num,config)